In [ ]:
#l2正则化
from tensorflow import keras
layer = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", kernel_regularizer=keras.regularizers.l2(0.01))
#l2（）函数返回一个正则化函数，在训练过程中的每个步骤都将调用该正则化函数来计算正则化损失。然后将其添加到最终损失中。如果需要l1正则化，可以只使用keras.regularizers.l1（）。如果你同时需要l1和l2正则化，请使用keras.regularizers.l1_l2（）

#为带有一些默认参数值的任何可调用对象创建一个小的包装函数
from functools import partial
RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax",
                     kernel_initializer="glorot_uniform")
])

In [ ]:
#dropout
#经过dropout训练的神经元不能与其相邻的神经元相互适应，它们必须自己发挥最大的作用。它们也不能过分依赖少数输入神经元，它们必须注意每个输入神经元。它们最终对输入的微小变化不太敏感。最后，你将获得一个更有鲁棒性的网络，该网络有更好的泛化能力。
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
#如果你发现模型过拟合，则可以提高dropout率。相反，如果模型欠拟合训练集，则应尝试降低dropout率。
#dropout确实会明显减慢收敛速度，但是如果正确微调，通常会导致更好的模型。

In [ ]:
#蒙特卡罗Dropout
import numpy as np
from tensorflow import keras
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X, y = make_classification(
    n_samples=1000,  # 总样本数
    n_features=20,   # 特征数
    n_classes=2,     # 二分类
    n_redundant=0,   # 无冗余特征
    n_informative=20, # 所有特征都有信息量
    random_state=42
)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp # 0.5 * 0.8 = 0.4 of total
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
y_probas = np.stack([model(X_test_scaled, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)
np.round(model.predict(X_test_scaled[:1]), 2) ,np.round(y_probas[:, :1], 2) ,np.round(y_proba[:1], 2)
#accuracy = np.sum(y_pred == y_test) / len(y_test)

#如果你的模型包含在训练过程中以特殊方式运行的其他层（例如BatchNormalization层），则应该使用以下MCDropout类[5]来替换Dropout层
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [ ]:
#最大范数正则化:不会把正则化损失项添加到总体损失函数中。取而代之的是，通常在每个训练步骤后通过计算||w||2来实现，如有需要，需要缩放w。
#减小r会增加正则化的数量，并有助于减少过拟合。最大范数正则化还可以帮助缓解不稳定的梯度问题

keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", kernel_constraint=keras.constraints.max_norm(1.))